# Digging into GRIB2 binary files (JMA landslide risk)

___
__Important:__ In this repository, we provide some scripts to handle the numerical landslide risk assessment figures prepared in 10-minute intervals by the Japan Meteorological Agency (JMA). This data is  *not freely available data*, but must rather be purchased from the Japan Meteorological Business Support Center (JMBSC), which operates under the oversight of the JMA. Here are a couple relevant links:

- Up to date visualized plot of landslide risk figures:<br> https://www.jma.go.jp/jp/doshamesh/

- JMBSC page for landslide risk data set:<br> http://www.jmbsc.or.jp/jp/online/file/f-online60210.html

<img src="img/dosha_example.jpeg" width="300">

This data is only provided in a standard binary format used frequently in meteorology, called *FM92 GRIB edition 2*. The software in this repository assumes that the user has already acquired such data via the appropriate channels.
___

## Opening and reading the GRIB file

To begin, let's clarify two points: (a) the nature of the data we are looking at; (b) the format of this data.

Regarding the nature of the data of interest, we are focusing exclusively on the landslide risk information computed and published by the JMA. The Japanese name is 土砂災害警戒判定メッシュ情報 (see links at the top of this page). The JMA assigns a numerical score to each point on a lattice, corresponding to the risk of landslides based on recent precipitation and accumulation of rainwater in the ground.

Regarding the format, this is technical but very straightforward. The data format is "General Representation of fields In Binary" (GRIB), and the specific guidelines are given in detail in the World Meteorological Organization's Manual on Codes. The JMA data that we will be examining here follows the FM92 GRIB edition 2, available at the WMO website in English (https://www.wmo.int/pages/prog/www/WMOCodes.html), and in Japanese from the JMA website (http://www.jma.go.jp/jma/kishou/books/tsuhoshiki/tsuhoshiki.html). The Japanese standards are based directly on this FM92 GRIB edition 2, and go by the name 国際気象通報式FM92 GRIB 二進形式格子点資料気象通報式(第2版).

There is really no choice but to go through the binary files one byte at a time to check everything out once; then everything can be easily automated. Useful resources when prying through these GRIB2 files are as follows:

- JMA Technical Report 374 (配信資料に関する技術情報（気象編）第374号). Available at:<br> http://www.data.jma.go.jp/add/suishin/jyouhou/pdf/374.pdf

- Description of the run-length encoding used for compression (ランレングス符号化法の解説). Available at:<br> http://www.jmbsc.or.jp/jp/online/c-onlineGsd.html

With these two resources in hand, we open the connection with an example file.

In [1]:
import os

toread = "demo_data/20130701/Z__C_RJTD_20130701000000_MET_INF_Jdosha_Ggis5km_ANAL_grib2.bin"

f_bin = open(toread, mode="rb")
print(f_bin)

<_io.BufferedReader name='demo_data/20130701/Z__C_RJTD_20130701000000_MET_INF_Jdosha_Ggis5km_ANAL_grib2.bin'>


The files are broken up into sections (節). We shall take one at a time.

## Section 0: 指示節

This section spans the first sixteen octets.

In [2]:
f_bin.seek(0)

0

In [3]:
b = f_bin.read(4) # Should be "GRIB".
print("bytes:", b)
s0_grib = str(b, 'ascii')
print(s0_grib)

bytes: b'GRIB'
GRIB


In [4]:
b = f_bin.read(2) # Should be missing.
print("bytes:", b)
s0_missing = b
print(s0_missing)

bytes: b'\xff\xff'
b'\xff\xff'


As regards the seventh byte, they refer to 符号表0.0, saying that it represents the meteorology field of interest.

In [5]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s0_field = int.from_bytes(b, byteorder="big", signed=False)
print(s0_field)

bytes: b'\x00'
0


In [6]:
b = f_bin.read(1) # Should be 2.
print("bytes:", b)
s0_version = int.from_bytes(b, byteorder="big", signed=False)
print(s0_version)

bytes: b'\x02'
2


The final handful of bytes give the *length of the GRIB message* (GRIB報全体の長さ). This is apparently variable from file to file.

In [7]:
b = f_bin.read(8) # Variable.
print("bytes:", b)
s0_griblen = int.from_bytes(b, byteorder="big", signed=False)
print(s0_griblen)

bytes: b'\x00\x00\x00\x00\x00\x00\x0e\xac'
3756


In [8]:
s0_seclen = 16 # hard-code for convenience.

___

## Section 1: 識別節

This section spans a total of twenty-one bytes.

In [9]:
f_bin.seek(s0_seclen)

16

In [10]:
b = f_bin.read(4) # Should be 21.
print("bytes:", b)
s1_seclen = int.from_bytes(b, byteorder="big", signed=False)
print(s1_seclen)

bytes: b'\x00\x00\x00\x15'
21


In [11]:
b = f_bin.read(1) # Should be 1.
print("bytes:", b)
s1_secnum = int.from_bytes(b, byteorder="big", signed=False)
print(s1_secnum)

bytes: b'\x01'
1


The next pair of octects are the "location where the data was created", I assume. They call it “作成中枢の識別”. This is identified via 共通符号表C-1. In any case, the value is supposed to be 34, which apparently corresponds to Tokyo.

In [12]:
b = f_bin.read(2) # Should be 34.
print("bytes:", b)
s1_center = int.from_bytes(b, byteorder="big", signed=False)
print(s1_center)

bytes: b'\x00"'
34


In [13]:
b = f_bin.read(2) # Should be 0.
print("bytes:", b)
s1_subcenter = int.from_bytes(b, byteorder="big", signed=False)
print(s1_subcenter)

bytes: b'\x00\x00'
0


The next three bytes are rather technical. First is the "version number of GRIB master table" (GRIBマスター表バージョン番号), second is "version number of GRIB region table" (GRIB地域表バージョン番号), and finally is the "meaning of the referred time (参照時刻の意味). Respectively, the meaning of these codes can be found in 符号表1.0, 1.1, and 1.2. Their expected values are 2, 1, and 0.

In [14]:
b = f_bin.read(1) # Should be 2.
print("bytes:", b)
s1_vermaster = int.from_bytes(b, byteorder="big", signed=False)
print(s1_vermaster)

bytes: b'\x02'
2


In [15]:
b = f_bin.read(1) # Should be 1.
print("bytes:", b)
s1_verregion = int.from_bytes(b, byteorder="big", signed=False)
print(s1_verregion)

bytes: b'\x01'
1


In [16]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s1_timemeaning = int.from_bytes(b, byteorder="big", signed=False)
print(s1_timemeaning)

bytes: b'\x00'
0


Next is some rather important metadata, namely the date/time, which has the meaning specified in the previous byte, which is apparently “解析”. The bytes are as follows:

- 2 bytes: year
- 1 byte: month
- 1 byte: day
- 1 byte: hour
- 1 byte: minute
- 1 byte: second

The interpretation of these values is specified in 「※１ 時刻表現」. Let's first read them out and take a look.

In [17]:
b = f_bin.read(2) # Should be the year.
print("bytes:", b)
s1_year = int.from_bytes(b, byteorder="big", signed=False)
print(s1_year)

bytes: b'\x07\xdd'
2013


In [18]:
b = f_bin.read(1) # Should be the month.
print("bytes:", b)
s1_month = int.from_bytes(b, byteorder="big", signed=False)
print(s1_month)

bytes: b'\x07'
7


In [19]:
b = f_bin.read(1) # Should be the day.
print("bytes:", b)
s1_day = int.from_bytes(b, byteorder="big", signed=False)
print(s1_day)

bytes: b'\x01'
1


In [20]:
b = f_bin.read(1) # Should be the hour.
print("bytes:", b)
s1_hour = int.from_bytes(b, byteorder="big", signed=False)
print(s1_hour)

bytes: b'\x00'
0


In [21]:
b = f_bin.read(1) # Should be the minute.
print("bytes:", b)
s1_minute = int.from_bytes(b, byteorder="big", signed=False)
print(s1_minute)

bytes: b'\x00'
0


In [22]:
b = f_bin.read(1) # Should be the second.
print("bytes:", b)
s1_second = int.from_bytes(b, byteorder="big", signed=False)
print(s1_second)

bytes: b'\x00'
0


Great. Very simple. This looks to match up directly with the file name (i.e., we could just parse the file name to get the same metadata).

The next byte is the document's status, either a "product" (現業プロダクト) or "test product" (現業的試験プロジェクト). Details are in 符号表1.3.

In [23]:
b = f_bin.read(1) # Should be 0 or 1.
print("bytes:", b)
s1_status = int.from_bytes(b, byteorder="big", signed=False)
print(s1_status)

bytes: b'\x00'
0


Finally, the document type, detailed in 符号表1.4, is supposedly 2, representing “解析及び予報プロダクト”.

In [24]:
b = f_bin.read(1) # Should be 2.
print("bytes:", b)
s1_type = int.from_bytes(b, byteorder="big", signed=False)
print(s1_type)

bytes: b'\x02'
2


___

## Section 2: 地域使用節

This section is apparently not used.

___

## Section 3: 格子系定義節

This section is for lattice definitions, and spans seventy-two bytes.

In [25]:
f_bin.seek((s0_seclen+s1_seclen))

37

In [26]:
b = f_bin.read(4) # Should be 72.
print("bytes:", b)
s3_seclen = int.from_bytes(b, byteorder="big", signed=False)
print(s3_seclen)

bytes: b'\x00\x00\x00H'
72


In [27]:
b = f_bin.read(1) # Should be 3.
print("bytes:", b)
s3_secnum = int.from_bytes(b, byteorder="big", signed=False)
print(s3_secnum)

bytes: b'\x03'
3


Reference for the lattice definition, detailed in 符号表3.0. Should be zero, with a meaning of “符号表3.1参照”, rather humorously.

In [28]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s3_defref = int.from_bytes(b, byteorder="big", signed=False)
print(s3_defref)

bytes: b'\x00'
0


Number of points on the lattice.

In [29]:
b = f_bin.read(4) # Should be 286720.
print("bytes:", b)
s3_numpoints = int.from_bytes(b, byteorder="big", signed=False)
print(s3_numpoints)

bytes: b'\x00\x04`\x00'
286720


Number of bytes used for a list that defines the number of points on the lattice (convoluted...), followed by the description of this list. Both are just zero here.

In [30]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s3_numpointsMem = int.from_bytes(b, byteorder="big", signed=False)
print(s3_numpointsMem)

bytes: b'\x00'
0


In [31]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s3_numpointsExp = int.from_bytes(b, byteorder="big", signed=False)
print(s3_numpointsExp)

bytes: b'\x00'
0


The next two bytes give the template number identifying the lattice definition type, details in 符号表3.1. It should be zero, meaning “緯度・経度格子”.

In [32]:
b = f_bin.read(2) # Should be 0.
print("bytes:", b)
s3_template = int.from_bytes(b, byteorder="big", signed=False)
print(s3_template)

bytes: b'\x00\x00'
0


Next is a key value: the identifier of the shape of the earth, details in 符号表3.2. This should be four, representing “GRS80回転楕円体”, recalling GRS80 is *Geodetic Reference System 1980*, a reference ellipsoid modelling the earth.

In [33]:
b = f_bin.read(1) # Should be 4.
print("bytes:", b)
s3_earthshape = int.from_bytes(b, byteorder="big", signed=False)
print(s3_earthshape)

bytes: b'\x04'
4


The next five are all missing.

In [34]:
b = f_bin.read(5) # Should be missing, likely max values.
print("bytes:", b)

bytes: b'\xff\xff\xff\xff\xff'


The next ten bytes are broken up as follows.

- 1 byte: major axis unit factor (尺度因子)
- 4 bytes: major axis length in given units (長軸の尺度付きの長さ)
- 1 byte: minor axis unit factor (尺度因子)
- 4 bytes: minor axis length in given units (短軸の尺度付きの長さ)

In [35]:
b = f_bin.read(1) # Should be 1.
print("bytes:", b)
s3_factormaj = int.from_bytes(b, byteorder="big", signed=False)
print(s3_factormaj)

bytes: b'\x01'
1


In [36]:
b = f_bin.read(4) # Should be 63781370.
print("bytes:", b)
s3_lenmaj = int.from_bytes(b, byteorder="big", signed=False)
print(s3_lenmaj)

bytes: b'\x03\xcd9\xfa'
63781370


In [37]:
b = f_bin.read(1) # Should be 1.
print("bytes:", b)
s3_factormin = int.from_bytes(b, byteorder="big", signed=False)
print(s3_factormin)

bytes: b'\x01'
1


In [38]:
b = f_bin.read(4) # Should be 63567523.
print("bytes:", b)
s3_lenmin = int.from_bytes(b, byteorder="big", signed=False)
print(s3_lenmin)

bytes: b'\x03\xc9\xf6\xa3'
63567523


Next, we have the absolute number of points on the lattice, following along the latitude and longitude lines. In Japanese, the former is “緯線に沿った格子点数”, the latter is “経線に沿った格子点数”.

In [39]:
b = f_bin.read(4) # Should be 512.
print("bytes:", b)
s3_numlon = int.from_bytes(b, byteorder="big", signed=False)
print(s3_numlon)

bytes: b'\x00\x00\x02\x00'
512


In [40]:
b = f_bin.read(4) # Should be 560.
print("bytes:", b)
s3_numlat = int.from_bytes(b, byteorder="big", signed=False)
print(s3_numlat)

bytes: b'\x00\x00\x020'
560


The interpretation as a grid is easy: we have a grid which is 512 cells wide, and 560 cells tall. As well, the number of cells computed here exactly equals the total number of points on the lattice we extracted earlier.

In [41]:
print(s3_numpoints)
print((s3_numlat*s3_numlon))

286720
286720


Next is some "base angle", should be zero.

In [42]:
b = f_bin.read(4) # Should be 0.
print("bytes:", b)
s3_baseangle = int.from_bytes(b, byteorder="big", signed=False)
print(s3_baseangle)

bytes: b'\x00\x00\x00\x00'
0


Next should be missing.

In [43]:
b = f_bin.read(4) # Should be 0.
print("bytes:", b)

bytes: b'\xff\xff\xff\xff'


Now are some numerical values we need to be careful with. The next eight bytes give the latitude (緯度) and longitude (経度) of the "first" point on the lattice. The units are in what they call “10-6度単位”, which is scientific notation for units re-scaled by $10^{-6}$. Let's see the values first, and then confirm we are handling what we are expecting.

In [44]:
b = f_bin.read(4) # Should be 47975000.
print("bytes:", b)
s3_firstlat = int.from_bytes(b, byteorder="big", signed=False)
print(s3_firstlat)

bytes: b'\x02\xdc\nX'
47975000


In [45]:
b = f_bin.read(4) # Should be 118031250.
print("bytes:", b)
s3_firstlon = int.from_bytes(b, byteorder="big", signed=False)
print(s3_firstlon)

bytes: b'\x07\t\x03\x92'
118031250


Note that if we multiply these values by $10^{-6}$, we have:

In [46]:
print((s3_firstlat/10**6))
print((s3_firstlon/10**6))

47.975
118.03125


This certainly look a lot more like degrees, and of course that is precisely what they are. The detailed calculations given in the JMA materials are as follows:

- First point's latitude: 48N - 3.0/60/2.
- First point's longitude: 118E + 3.75/60/2.

Of course, these calculations are easily confirmed:

In [47]:
print((48 - (3/(2*60))))
print((118 + (3.75/(2*60))))

47.975
118.03125


Next is a slightly complicated pair of terms: 分解能及び成分フラグ, with details in フラグ表3.3. The value should be 0x30, though it comes up as zero. In any case, the values after this guy are fine, so it seems there is no big issue.

In [48]:
b = f_bin.read(1) # Should be 0x30.
print("bytes:", b)

bytes: b'0'


Next, just as before, we have eight bytes dedicated to the latitude and longitude of the final lattice point/

In [49]:
b = f_bin.read(4) # Should be 20025000.
print("bytes:", b)
s3_lastlat = int.from_bytes(b, byteorder="big", signed=False)
print(s3_lastlat)

bytes: b'\x011\x8e\xa8'
20025000


In [50]:
b = f_bin.read(4) # Should be 149968750.
print("bytes:", b)
s3_lastlon = int.from_bytes(b, byteorder="big", signed=False)
print(s3_lastlon)

bytes: b'\x08\xf0Wn'
149968750


We can do analogous confirmations as before, although noting that the +/- signs in the shift are now opposite what they were for the first point.

- Last point's latitude: 20N + 3.0/60/2.
- Last point's longitude: 150E - 3.75/60/2.

Confirming everything:

In [51]:
print((s3_lastlat/10**6))
print((s3_lastlon/10**6))

20.025
149.96875


In [52]:
print((20 + (3/(2*60))))
print((150 - (3.75/(2*60))))

20.025
149.96875


Finally, we have the step size in the $i$-direction and $j$-direction respectively (called the $i$方向の増分, $j$方向の増分), defined:

- $i$-direction: 3.75/60
- $j$-direction: 3.0/60

The units, again, are $10^{-6}$ as before.

In [53]:
b = f_bin.read(4) # Should be 62500.
print("bytes:", b)
s3_isize = int.from_bytes(b, byteorder="big", signed=False)
print(s3_isize)

bytes: b'\x00\x00\xf4$'
62500


In [54]:
b = f_bin.read(4) # Should be 50000.
print("bytes:", b)
s3_jsize = int.from_bytes(b, byteorder="big", signed=False)
print(s3_jsize)

bytes: b'\x00\x00\xc3P'
50000


In [55]:
print((s3_isize/10**6))
print((s3_jsize/10**6))

0.0625
0.05


In [56]:
print(3.75/60)
print(3.0/60)

0.0625
0.05


As we would expect, the degrees *increase uniformly* (by the $i$ and $j$ sizes) over the lat/long directions on our graph.

In [57]:
tmp_val = s3_firstlon
print(tmp_val)
for i in range(s3_numlon-1):
    tmp_val += s3_isize
print(tmp_val)
print(s3_lastlon)

118031250
149968750
149968750


In [58]:
tmp_val = s3_firstlat
print(tmp_val)
for j in range(s3_numlat-1):
    tmp_val -= s3_jsize
print(tmp_val)
print(s3_lastlat)

47975000
20025000
20025000


Finally, we have the “走査モード”, namely a scanning mode, details in フラグ表3.4, and it should have a value of 0x00, namely zero.

In [59]:
b = f_bin.read(1) # Should be 0x00.
print("bytes:", b)

bytes: b'\x00'


___

## Section 4: プロダクト定義節

This section is for so-called "product" definitions, and spans forty-two bytes.

In [60]:
f_bin.seek((s0_seclen+s1_seclen+0+s3_seclen))

109

In [61]:
b = f_bin.read(4) # Should be 42.
print("bytes:", b)
s4_seclen = int.from_bytes(b, byteorder="big", signed=False)
print(s4_seclen)

bytes: b'\x00\x00\x00*'
42


In [62]:
b = f_bin.read(1) # Should be 4.
print("bytes:", b)
s4_secnum = int.from_bytes(b, byteorder="big", signed=False)
print(s4_secnum)

bytes: b'\x04'
4


The number of coordinate values immediately after the template (テンプレート直後の座標値の数). Should be zero.

In [63]:
b = f_bin.read(2) # Should be 0.
print("bytes:", b)
s4_coordnum = int.from_bytes(b, byteorder="big", signed=False)
print(s4_coordnum)

bytes: b'\x00\x00'
0


Product definition template number. Should be 50000, apparently with the meaning that the "product" here is based on processing other existing "products". Details in 符号表4.0.

In [64]:
b = f_bin.read(2) # Should be 50000.
print("bytes:", b)
s4_tempnum = int.from_bytes(b, byteorder="big", signed=False)
print(s4_tempnum)

bytes: b'\xc3P'
50000


Parameter category. Should be one, which means humidity (湿度), details in 符号表4.1.

In [65]:
b = f_bin.read(1) # Should be 1.
print("bytes:", b)
s4_paracat = int.from_bytes(b, byteorder="big", signed=False)
print(s4_paracat)

bytes: b'\x01'
1


Parameter number. Should be 208, which means landslide risk warnings (土砂災害警戒判定値), details in 符号表4.2. 

In [66]:
b = f_bin.read(1) # Should be 208.
print("bytes:", b)
s4_paranum = int.from_bytes(b, byteorder="big", signed=False)
print(s4_paranum)

bytes: b'\xd0'
208


Type of processing, details in 符号表4.3. Should be zero, meaning analysis/forecasting (解析及び予報).

In [67]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s4_proctype = int.from_bytes(b, byteorder="big", signed=False)
print(s4_proctype)

bytes: b'\x00'
0


Background processing method, details in 符号表JMA4.1, should be 160, meaning the specialized routine for landslide risk warnings (土砂災害警戒情報ルーチン).

In [68]:
b = f_bin.read(1) # Should be 160.
print("bytes:", b)
s4_backproc = int.from_bytes(b, byteorder="big", signed=False)
print(s4_backproc)

bytes: b'\xa0'
160


Next one is missing, the analysis method identifier.

In [69]:
b = f_bin.read(1) # Should be missing.
print("bytes:", b)

bytes: b'\xff'


Cut-off time measured from the reference time, in hours and minutes.

In [70]:
b = f_bin.read(2) # Should be 0.
print("bytes:", b)
s4_cutoffHour = int.from_bytes(b, byteorder="big", signed=False)
print(s4_cutoffHour)

bytes: b'\x00\x00'
0


In [71]:
b = f_bin.read(1) # Should be 10.
print("bytes:", b)
s4_cutoffMin = int.from_bytes(b, byteorder="big", signed=False)
print(s4_cutoffMin)

bytes: b'\n'
10


Type of units used for time period, details in 符号表4.4. Should be zero, meaning minutes.

In [72]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s4_timeunit = int.from_bytes(b, byteorder="big", signed=False)
print(s4_timeunit)

bytes: b'\x00'
0


Forecast time (予報時間). This is supposed to be zero.

In [73]:
b = f_bin.read(4) # Should be 0.
print("bytes:", b)
s4_fctime = int.from_bytes(b, byteorder="big", signed=False)
print(s4_fctime)

bytes: b'\x00\x00\x00\x00'
0


Type of "first fixed surface" (第一固定面の種類), with details in 符号表4.5. Should be one, meaning either land or water (地面または水面).

In [74]:
b = f_bin.read(1) # Should be 1.
print("bytes:", b)
s4_surftype = int.from_bytes(b, byteorder="big", signed=False)
print(s4_surftype)

bytes: b'\x01'
1


The next eleven bytes are all missing.

In [75]:
b = f_bin.read(1) # Should be missing.
print("bytes:", b)

bytes: b'\xff'


In [76]:
b = f_bin.read(4) # Should be missing.
print("bytes:", b)

bytes: b'\xff\xff\xff\xff'


In [77]:
b = f_bin.read(1) # Should be missing.
print("bytes:", b)

bytes: b'\xff'


In [78]:
b = f_bin.read(1) # Should be missing.
print("bytes:", b)

bytes: b'\xff'


In [79]:
b = f_bin.read(4) # Should be missing.
print("bytes:", b)

bytes: b'\xff\xff\xff\xff'


Name of the first set of related materials used in making this data set, details in 符号表JMA4.5, should be 2, meaning 土壌雨量指数の予測値. Otherwise, if the values are missing, it will be 255.

In [80]:
b = f_bin.read(1) # Should be 2 or 255.
print("bytes:", b)
s4_refsoil = int.from_bytes(b, byteorder="big", signed=False)
print(s4_refsoil)

bytes: b'\x02'
2


Difference between the *analysis* time and the reference time in the *current* referenced materials, in hours and minutes. By "current", we mean the materials identified in the previous octet. After this, there will be another set of reference materials, and of course in principle this could go on indefinitely, but here just two reference materials in total.

In [81]:
b = f_bin.read(2) # Should be variable.
print("bytes:", b)
s4_refsoilDiffHour = int.from_bytes(b, byteorder="big", signed=False)
print(s4_refsoilDiffHour)

bytes: b'\x00\x02'
2


In [82]:
b = f_bin.read(1) # Should be variable.
print("bytes:", b)
s4_refsoilDiffMin = int.from_bytes(b, byteorder="big", signed=False)
print(s4_refsoilDiffMin)

bytes: b'&'
38


Name of the second set of related materials used in making this data set, details in 符号表JMA4.5, should be 4, meaning 1時間降水量の予測値. Otherwise, if the values are missing, it will be 255.

In [83]:
b = f_bin.read(1) # Should be 4 or 255.
print("bytes:", b)
s4_refrain = int.from_bytes(b, byteorder="big", signed=False)
print(s4_refrain)

bytes: b'\x04'
4


Once again, time differences, now for the second set of reference materials.

In [84]:
b = f_bin.read(2) # Should be variable.
print("bytes:", b)
s4_refrainDiffHour = int.from_bytes(b, byteorder="big", signed=False)
print(s4_refrainDiffHour)

bytes: b'\x00\x00'
0


In [85]:
b = f_bin.read(1) # Should be variable.
print("bytes:", b)
s4_refrainDiffMin = int.from_bytes(b, byteorder="big", signed=False)
print(s4_refrainDiffMin)

bytes: b'\x00'
0


For reference, from the tenth byte through to the final byte of this section above, they say that "template 4.50000" is used.

___

## Section 5: 資料表現節

This section has "representations" of the main "materials". This is not the actual data, but rather the means of interpreting the data values which appear in the seventh section, a bit later.

In [86]:
f_bin.seek((s0_seclen+s1_seclen+0+s3_seclen+s4_seclen))

151

In [87]:
b = f_bin.read(4) # Should be variable.
print("bytes:", b)
s5_seclen = int.from_bytes(b, byteorder="big", signed=False)
print(s5_seclen)

bytes: b'\x00\x00\x00%'
37


In [88]:
b = f_bin.read(1) # Should be 5.
print("bytes:", b)
s5_secnum = int.from_bytes(b, byteorder="big", signed=False)
print(s5_secnum)

bytes: b'\x05'
5


Total number of data instances, should be equal to the number of lattice points seen earlier.

In [89]:
b = f_bin.read(4) # Should be 286720.
print("bytes:", b)
s5_numdata = int.from_bytes(b, byteorder="big", signed=False)
print(s5_numdata)

bytes: b'\x00\x04`\x00'
286720


Next is the template number of these materials, details in 符号表5.0. Should be 200, with meaning of 格子点資料 with run-length compression (ランレングス圧縮).

In [90]:
b = f_bin.read(2) # Should be 200.
print("bytes:", b)
s5_tempnum = int.from_bytes(b, byteorder="big", signed=False)
print(s5_tempnum)

bytes: b'\x00\xc8'
200


Next is the number of bits per individual datum.

In [91]:
b = f_bin.read(1) # Should be 8.
print("bytes:", b)
s5_databits = int.from_bytes(b, byteorder="big", signed=False)
print(s5_databits)

bytes: b'\x08'
8


Maximum "level" used in compression this time, a potentially variable value. Defined $V$ for later algebraic formulations.

In [92]:
b = f_bin.read(2) # Should be variable.
print("bytes:", b)
s5_V = int.from_bytes(b, byteorder="big", signed=False)
print(s5_V)

bytes: b'\x00\x04'
4


Maximum level, should be ten, and defined as $M$ for later use. Also, should have $V \leq M$.

In [93]:
b = f_bin.read(2) # Should be 10.
print("bytes:", b)
s5_M = int.from_bytes(b, byteorder="big", signed=False)
print(s5_M)

bytes: b'\x00\n'
10


Representative data value unit factor, should be zero.

In [94]:
b = f_bin.read(1) # Should be 0.
print("bytes:", b)
s5_datafactor = int.from_bytes(b, byteorder="big", signed=False)
print(s5_datafactor)

bytes: b'\x00'
0


Now, ranging over $m=1,2,\ldots,M$, we look at the data values that correspond to level $m$ (レベル$m$に対応するデータ代表値). Each of these uses two bytes. The interpretation is as follows (via ※2 in documentation):

- $m=0$. $R(m)$ is null. This is missing value.
- $m=1$. $R(m)=-2$. This is ocean or non-JPN lattice point.
- $m=2$. $R(m)=-1$. Lattice point on Japanese land, but not applicable for warnings.
- $m=3$. $R(m)=0$. This is warning level 0.
- $m=4$. $R(m)=1$. This is warning level 1.
- $m=5$. $R(m)=2$. This is warning level 2.
- $m=6$. $R(m)=3$. This is warning level 3.
- $m=7$. $R(m)=4$. This is warning level 4.
- $m=8$. $R(m)=5$. This is TBD.
- $m=9$. $R(m)=6$. This is TBD.
- $m=10$. $R(m)=7$. This is TBD.

Let's take a look at the first guys, one byte at a time.

In [95]:
b = f_bin.read(1)
print("bytes:", b)
b = f_bin.read(1)
print("bytes:", b)
b = f_bin.read(1)
print("bytes:", b)
b = f_bin.read(1)
print("bytes:", b)

bytes: b'\x80'
bytes: b'\x02'
bytes: b'\x80'
bytes: b'\x01'


Clearly, they're just using the byte value (in hex) of 0x80 to be a minus sign, as the second byte in each pair is precisely as we expect (2 and 1). Checking out the rest is easy.

In [96]:
for m in range(3,(s5_M+1)):
    b = f_bin.read(2)
    print("bytes:", b)
    tmp_level = int.from_bytes(b, byteorder="big", signed=True)
    print("m =", m, "; R(m) =", tmp_level)

bytes: b'\x00\x00'
m = 3 ; R(m) = 0
bytes: b'\x00\x01'
m = 4 ; R(m) = 1
bytes: b'\x00\x02'
m = 5 ; R(m) = 2
bytes: b'\x00\x03'
m = 6 ; R(m) = 3
bytes: b'\x00\x04'
m = 7 ; R(m) = 4
bytes: b'\x00\x05'
m = 8 ; R(m) = 5
bytes: b'\x00\x06'
m = 9 ; R(m) = 6
bytes: b'\x00\x07'
m = 10 ; R(m) = 7


___

## Section 6: ビットマップ節

This section is basically null, spanning just six bytes.

In [97]:
f_bin.seek((s0_seclen+s1_seclen+0+s3_seclen+s4_seclen+s5_seclen))

188

In [98]:
b = f_bin.read(4) # Should be 6.
print("bytes:", b)
s6_seclen = int.from_bytes(b, byteorder="big", signed=False)
print(s6_seclen)

bytes: b'\x00\x00\x00\x06'
6


In [99]:
b = f_bin.read(1) # Should be 6.
print("bytes:", b)
s6_secnum = int.from_bytes(b, byteorder="big", signed=False)
print(s6_secnum)

bytes: b'\x06'
6


Bitmap directive, should be 255, meaning that a bitmap isn't used here.

In [100]:
b = f_bin.read(1) # Should be 255.
print("bytes:", b)
s6_bitmap = int.from_bytes(b, byteorder="big", signed=False)
print(s6_bitmap)

bytes: b'\xff'
255


___

## Section 7: 資料節

Finally, the data section.

In [101]:
f_bin.seek((s0_seclen+s1_seclen+0+s3_seclen+s4_seclen+s5_seclen+s6_seclen))

194

In [102]:
b = f_bin.read(4) # Should be variable.
print("bytes:", b)
s7_seclen = int.from_bytes(b, byteorder="big", signed=False)
print(s7_seclen)

bytes: b'\x00\x00\r\xe6'
3558


In [103]:
b = f_bin.read(1) # Should be 7.
print("bytes:", b)
s7_secnum = int.from_bytes(b, byteorder="big", signed=False)
print(s7_secnum)

bytes: b'\x07'
7


Now comes a long series of data *values*, although these are "run-length compressed octects". The decompression is extremely straightforward once we know how the compression is done. Fortunately, this is explained in the second reference material linked at the start of this notebook.

In [104]:
bytes_left = s7_seclen - 5 # number of bytes left in this section.

In [105]:
# Before doing anything, read out the compressed data sequence.
data_compressed = []
bpd = s5_databits // 8 # bytes per data point.
for i in range(bytes_left):
    b = f_bin.read(bpd)
    #print("bytes:", b)
    b_int = int.from_bytes(b, byteorder="big", signed=False)
    data_compressed += [b_int]

In [106]:
# Key constants for decompression.
NBIT = s5_databits
MAXV = s5_V
LNGU = 2**NBIT - 1 - MAXV

In [107]:
print(NBIT, MAXV, LNGU)

8 4 251


In [108]:
import numpy as np

In [109]:
# Next, decompress this data sequence.
data_decompressed = []
dictpair = {"data": [], "runlen": []}
for i in range(len(data_compressed)):
    
    tocheck = data_compressed[i]
    print("tocheck =", tocheck, "; dict =", dictpair)
    
    if len(dictpair["data"]) == 0:
        dictpair["data"] = [tocheck]
    else:
        # If data slot is populated, then we're either dealing
        # with runlength, or moving to a new singleton.
        
        if tocheck > MAXV:
            # In this case, it is runlength data to be added.
            dictpair["runlen"] += [tocheck]
        else:
            # In this case, it is data, signifying a new set.
            # Thus, we must enter the "processing" routine.
            
            numrl = len(dictpair["runlen"])
            if numrl == 0:
                # If no runlength info, means it only appeared once.
                data_decompressed += dictpair["data"]
                #print("Adding to decompressed data:", dictpair["data"])
                
            else:
                # If there is runlength info, then must process it.
                rlinfo = np.array(dictpair["runlen"])
                rlexp = np.arange(numrl)
                rlvec = (LNGU**rlexp) * (rlinfo - (1+MAXV))
                rl = np.sum(rlvec) + 1
                data_decompressed += rl * dictpair["data"]
                #print("Adding to decompressed data:", rl * dictpair["data"])
            
            # Reset, critically noting that we have a new data point to store.
            dictpair["data"] = [tocheck]
            dictpair["runlen"] = []

# Finally, note that there will ALWAYS be something left over.
if tocheck <= MAXV:
    # If the last guy was small, then just add it.
    data_decompressed += dictpair["data"]
    #print("Adding to decompressed data:", dictpair["data"])
else:
    # If the last guy was large, we have non-trivial run-length.
    numrl = len(dictpair["runlen"])
    rlinfo = np.array(dictpair["runlen"])
    rlexp = np.arange(numrl)
    rlvec = (LNGU**rlexp) * (rlinfo - (1+MAXV))
    rl = np.sum(rlvec) + 1
    data_decompressed += rl * dictpair["data"]
    #print("Adding to decompressed data:", rl * dictpair["data"])

tocheck = 1 ; dict = {'data': [], 'runlen': []}
tocheck = 123 ; dict = {'data': [1], 'runlen': []}
tocheck = 106 ; dict = {'data': [1], 'runlen': [123]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [123, 106]}
tocheck = 6 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [6]}
tocheck = 248 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [248]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [248, 6]}
tocheck = 6 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [6]}
tocheck = 16 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [16]}
tocheck = 8 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [8]}
tocheck = 247 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [247]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [247, 6]}
tocheck = 7 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data

tocheck = 204 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [204]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [204, 6]}
tocheck = 64 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [64]}
tocheck = 203 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [203]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [203, 6]}
tocheck = 64 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [64]}
tocheck = 202 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [202]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [202, 6]}
tocheck = 66 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [66]}
tocheck = 203 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [203]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [203, 6]}
tocheck = 65 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; d

tocheck = 77 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [77]}
tocheck = 145 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [145]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [145, 6]}
tocheck = 9 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [9]}
tocheck = 46 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [46]}
tocheck = 77 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [77]}
tocheck = 146 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [146]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [146, 6]}
tocheck = 7 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [7]}
tocheck = 47 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [47]}
tocheck = 77 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3

tocheck = 9 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [9]}
tocheck = 15 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [15]}
tocheck = 9 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [9]}
tocheck = 138 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [138]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [138, 6]}
tocheck = 106 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [106]}
tocheck = 7 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [7]}
tocheck = 11 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [11]}
tocheck = 13 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [13]}
tocheck = 9 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [9]}
tocheck = 137 ; dict = {'data': [1], 'ru

tocheck = 1 ; dict = {'data': [3], 'runlen': [8]}
tocheck = 6 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [6]}
tocheck = 9 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [9]}
tocheck = 6 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [6]}
tocheck = 26 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [26]}
tocheck = 13 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [13]}
tocheck = 6 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [6]}
tocheck = 6 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [6]}
tocheck = 1 ; dict = {'data': [3], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [6]}
tocheck = 6 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [6]}
tochec

tocheck = 11 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [11]}
tocheck = 7 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [7]}
tocheck = 9 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [9]}
tocheck = 6 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [6]}
tocheck = 7 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [7]}
tocheck = 245 ; dict = {'data': [1], 'runlen': []}
tocheck = 6 ; dict = {'data': [1], 'runlen': [245]}
tocheck = 3 ; dict = {'data': [1], 'runlen': [245, 6]}
tocheck = 12 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [12]}
tocheck = 6 ; dict = {'data': [1], 'runlen': []}
tocheck = 3 ; dict = {'data': [1], 'runlen': [6]}
tocheck = 10 ; dict = {'data': [3], 'runlen': []}
tocheck = 1 ; dict = {'data': [3], 'runlen': [10]}
tocheck = 250 ; dict = {'data': [1], 'runlen

In [110]:
print(len(data_decompressed))
print(np.unique(np.array(data_decompressed)))

286720
[0 1 3 4]


In [111]:
#print(data_decompressed)

In [112]:
print(s3_numpoints)
#print(512*560)
#print(s3_numlat)
#print(s3_numlon)

286720


In [113]:
s3_numpoints-len(data_decompressed)

0

If this value is zero, we can be essentially certain that we have successfully decoded the compressed data.

___

## Section 8: 終端節

Basically a trivial final section that lets us know we're finished.

In [114]:
f_bin.seek((s0_seclen+s1_seclen+0+s3_seclen+s4_seclen+s5_seclen+s6_seclen+s7_seclen))

3752

In [115]:
b = f_bin.read(4) # Should be "7777".
print(b)
s8_grib = str(b, 'ascii')
print(s8_grib)

b'7777'
7777


In [116]:
b = f_bin.read(1) # Should be at the end...
print(b)

b''


___